In [8]:
import os
import time
import cv2
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from queue import Queue
import threading
from PIL import Image

from ultralytics import YOLO

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

frame_count = 0
start_time = time.time()

num_classes = 36
learning_rate = 0.001

transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.Grayscale(),
    transforms.ToTensor()
])

folders = os.listdir(r'C:\Users\Ilham Mafani Nadif\Videos\Deteksi Plat Nomor\karakter\train')
array_label = np.array(folders)

model_yolo = YOLO(r'C:\Users\Ilham Mafani Nadif\Videos\Deteksi Plat Nomor\Plat Nomor Luar\runs\detect\train6\weights\best.pt')
video_path = r'C:\Users\Ilham Mafani Nadif\Videos\Deteksi Plat Nomor\Implementasi\Tengah.mp4'
model_path = r'C:\Users\Ilham Mafani Nadif\Videos\Deteksi Plat Nomor\fix_model_cnn\last_model.pth'

image_queue = Queue()
segment_queue = Queue(maxsize=0)

class CNN(nn.Module):
    def __init__(self, num_classes):
        super(CNN, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        
        self.classifier = nn.Sequential(
            nn.Linear(32 * 16 * 16, 128),
            nn.ReLU(),
            nn.Linear(128, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

model = CNN(num_classes).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

def inference(image):
    image = transform(image).unsqueeze(0).to(device)
        
    model_inference = CNN(num_classes).to(device)
    model_inference.load_state_dict(torch.load(model_path))
    model_inference.eval()

    with torch.no_grad():
        output = model_inference(image)
        _, predicted = torch.max(output.data, 1)
        return predicted.item()

def process_image():
    while True:
        image_path, image = image_queue.get()
        cv2.imwrite(image_path, image)
        image_queue.task_done()

def segmentation(image):
    contours, _ = cv2.findContours(image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    outer_contour = max(contours, key=cv2.contourArea)
    x, y, w, h = cv2.boundingRect(outer_contour)
    cropped_image = image[y:y+h, x:x+w]
    return cropped_image


def segment_char(image, timestamp):
    img_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    binary_img = cv2.adaptiveThreshold(img_gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2)
    total_black_pixels = np.sum(binary_img == 0)
    total_white_pixels = np.sum(binary_img == 255)

    binary_img = cv2.adaptiveThreshold(img_gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2)
    total_black_pixels = np.sum(binary_img == 0)
    total_white_pixels = np.sum(binary_img == 255)

    if total_black_pixels > total_white_pixels:
        print('plat hitam')
        binary_img = binary_img
    else:
        print('plat putih')
        binary_img = ~binary_img

    eroded = binary_img
    contours, _ = cv2.findContours(eroded, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    contours = sorted(contours, key=lambda c: cv2.boundingRect(c)[0])
    
    image_with_boxes = cv2.cvtColor(eroded, cv2.COLOR_GRAY2RGB).copy()
    image_with_boxes_ori = image.copy()
    cropped_image = np.zeros_like(eroded)
    
    output_folder = "fps_output/segmented_images"
    os.makedirs(output_folder, exist_ok=True)
    
    output_folder_1 = "fps_output/box_images"
    os.makedirs(output_folder_1, exist_ok=True)

    output_folder_2 = "fps_output/result"
    os.makedirs(output_folder_2, exist_ok=True)

    predictions = []

    hx = eroded.shape[0]
    
    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        if h > (1/3 * hx) and h < (1/2 * hx) and w < (1/2 * h + 1/4*h):
            cv2.rectangle(image_with_boxes, (x, y), (x + w, y + h), (0,255,0), 1)
            cropped_image[y:y+h, x:x+w] = eroded[y:y+h, x:x+w]
                            
            array_segmented_image = Image.fromarray(cropped_image[y:y+h, x:x+w])
            predicted_label = inference(array_segmented_image)
            labels = array_label[predicted_label]

            cv2.rectangle(image_with_boxes_ori, (x, y), (x + w, y + h), (255,0,0), 1)
            cv2.putText(image_with_boxes_ori, labels, (x + w - 9, y + h + 8), cv2.FONT_HERSHEY_SIMPLEX, 0.3, (0,0,255), 1)
                            
            predictions.append(labels)
            result = ''.join(predictions)

            image_name = f"{result}___{timestamp}"
            image_path = os.path.join(output_folder, image_name)
            box_path = os.path.join(output_folder_1, image_name)
            ori_path = os.path.join(output_folder_2, image_name)

            target_width = 300
            height, width = cropped_image.shape[:2]
            aspect_ratio = target_width / width
            target_height = int(height * aspect_ratio)

            resized_biner = cv2.resize(cropped_image, (target_width, target_height), interpolation=cv2.INTER_NEAREST)
            resized_bbox = cv2.resize(image_with_boxes, (target_width, target_height), interpolation=cv2.INTER_NEAREST)
            resized_ori = cv2.resize(image_with_boxes_ori, (target_width, target_height), interpolation=cv2.INTER_NEAREST)

            cv2.imwrite(box_path, resized_bbox)
            cv2.imwrite(image_path, resized_biner)
            cv2.imwrite(ori_path, resized_ori)

def segment_image():
    while True:
        image, timestamp = segment_queue.get()
        segment_char(image, timestamp)
        segment_queue.task_done()

cap = cv2.VideoCapture(video_path)

line_coordinates = [(150, 400), (1300, 300)] 
stop_inference = False

output_folder = "fps_output/plat_nomor"
os.makedirs(output_folder, exist_ok=True)

output_frames = "fps_output/frames"
os.makedirs(output_frames, exist_ok=True)

for _ in range(3):
    t = threading.Thread(target=process_image)
    t.daemon = True
    t.start()

for _ in range(1):
    t = threading.Thread(target=segment_image)
    t.daemon = True
    t.start()
    
while cap.isOpened() and not stop_inference:
    success, frame = cap.read()

    if success:
        result = model_yolo.predict(frame, imgsz=640, conf=0.5, iou=0.9)
        annotated_frame = result[0].plot()
        
        boxes = result[0].boxes
        list = boxes.xyxy.tolist()
        rectangles = [(int(x1), int(y1), int(x2), int(y2)) for x1, y1, x2, y2 in list]

        line_start = (line_coordinates[0][0], line_coordinates[0][1])
        line_end = (line_coordinates[1][0], line_coordinates[1][1])
        cv2.line(annotated_frame, line_start, line_end, (0, 0, 255), 2)
        frame_count += 1
        for i, (x1, y1, x2, y2) in enumerate(rectangles):
            if y1 < line_start[1] and y2 > line_end[1]:
                cropped_image = frame[y1:y2, x1:x2]
                h, w, c = cropped_image.shape
                if w > 2 * h and h < w:
                    timestamp = time.strftime("%Y%m%d%H%M%S", time.localtime())  # Current timestamp
                    names = f"{timestamp}_{i}.jpg"

                    image_name = f"detected___" + names
                    image_path = os.path.join(output_folder, image_name)

                    frame_name = f"frames___" + names 
                    frames_path = os.path.join(output_frames, frame_name)
                    image_queue.put((frames_path, annotated_frame))
                    image_queue.put((image_path, cropped_image))
                    segment_queue.put((cropped_image, names))
                    
                    
        cv2.imshow("Deteksi Plat Nomor", annotated_frame)
        if cv2.waitKey(1) & 0xFF == ord("q"):
            stop_inference = True
            break
    else:
        break

end_time = time.time()
cap.release()
cv2.destroyAllWindows()

elapsed_time = end_time - start_time
fps = frame_count / elapsed_time
print("FPS:", fps)

cuda



0: 384x640 (no detections), 68.9ms
Speed: 2.0ms preprocess, 68.9ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 (no detections), 10.9ms
Speed: 0.5ms preprocess, 10.9ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 (no detections), 10.9ms
Speed: 1.0ms preprocess, 10.9ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 (no detections), 9.4ms
Speed: 0.5ms preprocess, 9.4ms inference, 1.5ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 (no detections), 8.9ms
Speed: 0.5ms preprocess, 8.9ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 (no detections), 8.4ms
Speed: 1.0ms preprocess, 8.4ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 (no detections), 9.4ms
Speed: 0.5ms preprocess, 9.4ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 (no detections), 8.9ms
Speed: 0.5ms preprocess, 8.9ms inference, 

plat putih
plat putih
plat hitam
plat hitam


Speed: 0.5ms preprocess, 12.9ms inference, 5.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 2 plat_nomors, 14.9ms
Speed: 1.5ms preprocess, 14.9ms inference, 3.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 1 plat_nomor, 8.9ms
Speed: 1.0ms preprocess, 8.9ms inference, 5.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 2 plat_nomors, 15.9ms
Speed: 0.5ms preprocess, 15.9ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 1 plat_nomor, 15.4ms
Speed: 0.5ms preprocess, 15.4ms inference, 2.5ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 2 plat_nomors, 10.4ms
Speed: 0.0ms preprocess, 10.4ms inference, 3.5ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 1 plat_nomor, 18.4ms


plat hitam
plat hitam
plat hitam
plat hitam


...

0: 384x640 (no detections), 13.4ms
Speed: 0.5ms preprocess, 13.4ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 (no detections), 12.4ms
Speed: 0.5ms preprocess, 12.4ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 (no detections), 16.4ms
Speed: 0.5ms preprocess, 16.4ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 (no detections), 13.4ms
Speed: 0.5ms preprocess, 13.4ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 (no detections), 12.4ms
Speed: 0.5ms preprocess, 12.4ms inference, 1.5ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 (no detections), 13.9ms
Speed: 0.5ms preprocess, 13.9ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 (no detections), 9.4ms
Speed: 0.5ms preprocess, 9.4ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 (no detections), 12.4ms
Speed: 0.5ms preprocess, 12.4ms infe

plat putih
plat putih


Speed: 1.0ms preprocess, 9.9ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 (no detections), 13.4ms
Speed: 0.0ms preprocess, 13.4ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)


FPS: 47.97482522895812


plat putih
plat putih
